# Advanced Operating Systems: Lab 1 - Getting started with kernel tracing - I/O

This Jypterlab Notebook is intended to get you started with:

1. Building and running the benchmark
2. Extracting and plotting data collected by the benchmark itself (e.g., execution time)
3. Extracting data collected externally by DTrace

This file is not intended to be a template for your solutions; we recommend that you create a new Notebook, placing your work there, copying and pasting template code from this Notebook as seems useful.

You can add new cells to your notebook using the '+' button on the panel above.  You can change whether a cell is treated as Python or Markdown using the selector above (it defaults to 'Code').

Make sure to run cells in the right order (pressing Ctrl-Enter when in the cell to run it) so that dependencies are executed in the right order. For example, Python imports must occur before running the remainder of the code, and data must be collected before it can be plotted.  If you restart Jupyterlab, the Python kernel will restart, and one-time Python intialisation must be performed again (e.g., imports and data collection).

**Note**: When you execute a cell in Jupyterlab, the bracketed number to the left (e.g., `[1]`) will temporarily change to a `[*]` to indicate that it has not yet completed executin. Running benchmarks or longer forms of data analysis or plotting may take a considerable time on our RPi4 boards, so do exercise patience.

# 1. Building and running the benchmark

## Building the benchmark

First, using Jupyter's `!` syntax, build the benchmark using `make`; if no dependencies have changed, you may get no output:

In [ ]:
!make -C io

## The benchmark command line

Next, we can run the benchmark tool without command-line arguments to get usage information:

In [ ]:
!io/io-benchmark

**Note**: Not all of the benchmark modes and options are relevant to this year's assignment.

## Create the benchmark data file (one time only)

Now, create a data file suitable for the I/O benchmark to use. This needs to be done only once. Use a total file size of 16MiB:

In [ ]:
!io/io-benchmark -t 16777216 create iofile

## Describe the hardware, OS, and default benchmark configuration

The benchmark tool's `describe` command provides more detailed about the hardware, OS, and benchmark configuration. This information will also be generated by the other commands if you specify `-v` with them, which can be useful to capture duriing benchmark runs to validate the configuration or use as input to data analysis or plotting.

In [ ]:
!io/io-benchmark describe iofile

## Machine-readable output

With the `-j` option, the benchmark tool will generate all output including configuration information and benchmark results as JSON. Using Python JSON support (e.g., `json.loads`), you can avoid manual parsing to analyse data in Python:

In [ ]:
!io/io-benchmark -j describe iofile

## Example benchmark run

This command line runs the benchmark with a block size of 256KiB (`-b 262144`), total I/O size of 16MiB (`-t 16777216`), captures rusage information (`-g`), prints the results in JSON (`-j`), includes additional configuration information (`-v`), runs two iterations (`-n 2`), and executes a read workload (`read`) on the previously created data file `iofile`)

In [ ]:
!io/io-benchmark -b 262144 -t 16777216 -g -j -v -n 2 read iofile

The `"hw_configuration"`, `"os_configuration"`, and `"benchmark_configuration"` blocks provide information about the configuration of the host and the benchmark as above.  Remove `-v` from the command line to omit this information from benchmark runs.

The `"benchmark_samples"` block consists of an array of individual measurement results with various metrics for each measurement. In general, dropping the first sample is a good idea, as it may contain artifacts from "first runs" -- such as the one-time costs of lazy dynamic linking. The captured metrics using this benchmark command line include:

- `bandwidth`: The average bandwidth over the run of the benchmark's work loop.
- `time`: Wall-clock time running the work loop.
- `utime` and `stime`: Sampled user and system (kernel) time. This may not add up to wall-clock time if software has to sleep awaiting I/O. Further, while `time` is measured using precise clock reads, `utime` and `stime` are sampled by the timer interrupt. You therefore cannot expect that (`time` == `utime` + `stime`).
- `inblock` and `outblock`: The number of actual block I/O operations performed by the process measured using `getrusage(2)`.

# 2. Generating, extracting and plotting data generated by the benchmark

## Initialise Python

First, import Python module dependencies and set configuration parameters. This needs to be done once per session:

In [ ]:
import json
# Enable Jupyter notebook mode for matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Set low for experimentation; consider using 7 "in production", but that will take a lot longer!
iterations=3
logminbufsize=5
logmaxbufsize=25

buffersizes = [2**v for v in range(logminbufsize, logmaxbufsize)]

## Run benchmark and collect results in Python data structures

Next, run the benchmark and digest the results. We run the benchmark binary once for each buffer size, requesting it execute `iterations` iterations each time, generating JSON output. We import the JSON into Python, and generate some summary statistics (medians and quartiles) for each buffer size. In this example, we consider only bandwidth, but you can also easily plot properties such as I/O counts or user/system time (including distribution properties such as IQR).

**Note**: You will likely want to modify this code to drop the first sample for each size.

In [ ]:
benchmark_strings = {}
print("Benchmark run starting")
for buffersize in buffersizes:
    print("Buffer size: ", buffersize)
    output = !io/io-benchmark -b $buffersize -t 16777216 -j -n $iterations -v read iofile
    benchmark_strings[buffersize] = ' '.join(output)
print("Benchmark run completed")
    
bw_samples = {}       # Arrays of bandwidth samples indexed by buffer size
medians = {}          # Arrays of medians indexed by buffer size
stds = {}             # Arrays of standard deviations indexed by buffer size
q3s = {}              # Arrays of third quartiles
q1s = {}              # Arrays of first quartiles

for buffersize in buffersizes:
    j = json.loads(benchmark_strings[buffersize])
    samples = list(j["benchmark_samples"])
    bw_samples[buffersize] = list([x["bandwidth"] for x in samples])
    medians[buffersize] = np.median(bw_samples[buffersize])
    q1s[buffersize] = medians[buffersize] - np.quantile(bw_samples[buffersize], 0.25)
    q3s[buffersize] = np.quantile(bw_samples[buffersize], 0.75) - medians[buffersize] 

## Plot the collected data

Finally, we generate a plot using `matplotlib`, consisting of medians and error bars based on IQR:

In [ ]:
fig1, ax = plt.subplots()
ax.set_title("buffer size vs. bandwidth")

x_coords = []
y_coords = []
low_errs = []
high_errs = []

for x in buffersizes:
    x_coords.append(x)
    y_coords.append(medians[x])
    low_errs.append(q1s[x])
    high_errs.append(q3s[x])

ax.set_xscale("log")
ax.errorbar(x_coords, y_coords, [low_errs, high_errs])
plt.show()

## Create an annotated plot

In analysing this plot, it is worth considering key inflection points: Points on the plot where there are behavioural changes, and what they reflect. We can directly annotate those points on the plot using `avxline`.

In the next plot, we've manually placed several vertical lines at OS and microarchitectural thresholds where the data you collect might experience inflection points.

Be sure to take note of the linear Y axis and exponential X axis, and consider its implications for data analysis.

In [ ]:
### This content the same as the above cell
fig1, ax = plt.subplots()
ax.set_title("buffer size vs. bandwidth")

x_coords = []
y_coords = []
low_errs = []
high_errs = []

for x in buffersizes:
    x_coords.append(x)
    y_coords.append(medians[x])
    low_errs.append(q1s[x])
    high_errs.append(q3s[x])

ax.set_xscale("log")
ax.errorbar(x_coords, y_coords, [low_errs, high_errs])

### This is new content relative to the prior cell
ax.axvline(x=4*1024, color="red", label="4KB", linestyle=":")
ax.axvline(x=64*1024, color="blue", label="64KB", linestyle=":")
ax.axvline(x=128*1024, color="green", label="128KB", linestyle=":")
ax.legend()
ax.errorbar(x_coords, y_coords, [low_errs, high_errs])
plt.show()

You can save a plot out to disk as a PDF -- e.g., for use in a lab report -- using this API, which should be called just prior to `plt.show()`:

In [ ]:
#plt.savefig("performance.pdf")

# 3. Extracting and plotting data generated using DTrace

DTrace scripts can be run directly from Python and Jupyter, returning a data structure that describes the resulting output. The details of the data structure depend on the script you have written. The DTrace script will run asynchronously while the benchmark runs, and you then collect the data after completion.

You will likely wish to develop DTrace scripts using the `dtrace(1)` command-line tool rather than in Python, as that will give more ready access to debug output (such as script compilation failure details).

## Import the DTrace module

First import the `python-dtrace` module; this must be done once per session:

In [ ]:
from dtrace import DTraceConsumerThread

## Collect system-call data

The following example uses DTrace to record the number of times each `syscall` is called by the `io-benchmark` benchmark reading the benchmark data file. Note that it brackets data collection based on both the executable name (`io-benchmark`) and also the start and finish of the benchmark loop as detected using calls to the `clock_gettime(2)` system call (note that the system call is invoked directly in `io-benchmark` to bypass `vdso` optimisation). We set iterations to 1 to avoid capturing data from more than one run. When you run experiments in the lab, you will want to run this for each applicable buffer size, in order to produce plots similar to the above:

In [ ]:
# Sample D-language script embedded in a Python string.
# Note that the 'done' variable assumes one benchmark iteration.
# You will need to extend this script to use it with multiple iterations.
io_syscall_script = """

/*
 * Instrument clock_gettime(2) system-call return.  We will use this to detect
 * the start of the benchmark's work loop.
 */
syscall::clock_gettime:return
/execname == "io-benchmark" && !self->in_benchmark && !self->done/
{
    self->in_benchmark = 1;
}

/*
 * Intrument clock_gettime() system-call entry.  We will use this to detect the
 * end of the benchmark's work loop.
 */
syscall::clock_gettime:entry
/execname == "io-benchmark" && self->in_benchmark  && !self->done/
{
    self->in_benchmark = 0;
    self->done = 1;
}

/*
 * Instrument entry to every system call except clock_gettime(2), capturing a
 * count of the number of times each call is made.
 */
syscall:::entry
/execname == "io-benchmark" && self->in_benchmark && probefunc != "clock_gettime"/
{
    @a[probefunc] = count();
}
"""

from collections import defaultdict
values = defaultdict(int)

# Python-DTrace executes the DTrace script asynchronously in another thread.
# It uses a set of possible callbacks to return aggregation data to Python.
def simple_walk(action, identifier, keys, value):
    """
    action -- type of action (sum, avg, ...)
    identifier -- the id.
    keys -- list of keys.
    value -- the value.
    """
    values[keys[0]] += value

# Create a seperate thread to run the DTrace instrumentation
dtrace_thread = DTraceConsumerThread(io_syscall_script,
                                     walk_func=simple_walk,
                                     out_func=lambda v: None,
                                     chew_func=lambda v: None,
                                     chewrec_func=lambda v: None,
                                     sleep=1)

# Start the DTrace instrumentation
dtrace_thread.start()

# Display header to indicate that the benchmarking has started
print("Starting io-benchmark read performance measurement")

# Run the io-benchmark benchmark    
BUFFER_SIZE = 512

output_dtrace = !io/io-benchmark -b {str(BUFFER_SIZE)} read iofile
        
# The benchmark has completed - stop the DTrace instrumentation
dtrace_thread.stop()
dtrace_thread.join()
    
# Print the syscalls and their frequency
for x in values.keys():
    print("Number of ", x, " calls: ", values[x])

# Display footer to indicate that the benchmarking has finished
print("Finished io-benchmark read performance measurement")

This approach can be used to extract a variety of kernel trace data using DTrace. One known limitation is that stack() results are stored as a set of code addresses, rather than being expanded to strings, so you will likely prefer to use the `dtrace(1)` command-line tool to capture stack data.